<a href="https://colab.research.google.com/github/sanjanasrinivas22/1BM23CS301-AI/blob/main/FOLFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import re

print("Sanjana Srinivas-1BM23CS301")




# --------------------- Pattern Matching ---------------------

def match_pattern(pattern, fact):
    """
    Matches a rule pattern (like Sells(p, q, r)) with a fact (like Sells(Robert, T1, A)).
    Returns substitution dictionary if matched, else None.
    """
    # Extract predicate and arguments
    pat_pred, pat_args = re.match(r'(\w+)\(([^)]*)\)', pattern).groups()
    fact_pred, fact_args = re.match(r'(\w+)\(([^)]*)\)', fact).groups()

    # Predicate names must match
    if pat_pred != fact_pred:
        return None

    pat_args = [a.strip() for a in pat_args.split(",")]
    fact_args = [a.strip() for a in fact_args.split(",")]

    if len(pat_args) != len(fact_args):
        return None

    subst = {}
    for p_arg, f_arg in zip(pat_args, fact_args):
        # Variables are lowercase identifiers (e.g., p, x, y)
        if re.fullmatch(r'[a-z]\w*', p_arg):
            subst[p_arg] = f_arg
        elif p_arg != f_arg:
            return None
    return subst


def apply_substitution(expr, subst):
    """
    Replaces all variable names in an expression using the given substitution dictionary.
    Example: apply_substitution("Sells(p,q,r)", {'p':'Robert','q':'T1','r':'A'})
             -> "Sells(Robert,T1,A)"
    """
    for var, val in subst.items():
        expr = re.sub(rf'\b{var}\b', val, expr)
    return expr


# --------------------- Knowledge Base ---------------------

rules = [
    (["American(p)", "Weapon(q)", "Sells(p,q,r)", "Hostile(r)"], "Criminal(p)"),
    (["Missile(x)"], "Weapon(x)"),
    (["Enemy(x, America)"], "Hostile(x)"),
    (["Missile(x)", "Owns(A, x)"], "Sells(Robert, x, A)")
]

facts = {
    "American(Robert)",
    "Enemy(A, America)",
    "Owns(A, T1)",
    "Missile(T1)"
}

goal = "Criminal(Robert)"


# --------------------- Forward Chaining ---------------------

def forward_chain(rules, facts, goal):
    """
    Applies forward chaining to derive new facts from given rules and initial facts.
    Returns True if goal is achieved.
    """
    added = True
    while added:
        added = False
        for premises, conclusion in rules:
            possible_substs = []

            # Check each premise of the rule
            for p in premises:
                matched = False
                for f in facts:
                    subst = match_pattern(p, f)
                    if subst:
                        possible_substs.append(subst)
                        matched = True
                        break
                if not matched:
                    break  # one premise fails → skip rule
            else:
                # Combine substitutions into one dictionary
                combined = {}
                for s in possible_substs:
                    combined.update(s)

                new_fact = apply_substitution(conclusion, combined)

                if new_fact not in facts:
                    facts.add(new_fact)
                    print(f"Inferred: {new_fact}")
                    added = True
                    if new_fact == goal:
                        return True
    return goal in facts




print("Goal achieved:", forward_chain(rules, facts, goal))


Sanjana Srinivas-1BM23CS301
Inferred: Weapon(T1)
Inferred: Hostile(A)
Inferred: Sells(Robert, T1, A)
Inferred: Criminal(Robert)
Goal achieved: True
